In [73]:
# importing libraries
import nltk
import pandas as pd
import numpy as np
import ast
import json
import re
# import swifter
from tqdm import tqdm
import time
from nltk.stem.wordnet import WordNetLemmatizer
from PyDictionary import PyDictionary # for dictionary splitting

In [74]:
# reading dataframes

def read_file(summary_file_path, tech_file_path, output_file_path):

    df_summary = pd.read_csv(summary_file_path)
    df_tech = pd.read_csv(tech_file_path)
    df_output = pd.read_csv(output_file_path)
    
    return df_summary, df_tech, df_output

In [75]:
!pwd

/Users/shubhamsunwalka/Desktop/Slintel_2/training_spacy


In [76]:
summary_file_path = "../tech_sort1k.csv"
tech_file_path = "../Technology Summary Matching Project - technologies-post-removing.csv"
# tech_file_path = "../Tech_pydictionary.csv"
output_file_path = "../Dataset/output - output.csv"

In [77]:
summary, tech, output = read_file(summary_file_path, tech_file_path, output_file_path)

In [78]:
print("\n Summary Data \n", summary.head())
print("\n Technology Sheet Data \n", tech.head())
print("\n output Data \n", output.head())


 Summary Data 
                             id exact_matched_patt_contextual  \
0  zsDSYc5UzpyXekOABZBfnQ_0000                           NaN   
1  zs1YNrrCeorFkGLEXSSp-A_0000                           NaN   
2  zowirq8ZhxhchApaRMhNSA_0000                           NaN   
3  zoQVkPCfXB9n3AaPGHHwzg_0000                           NaN   
4  zmIr2glBZ3Ef8CSS0jw1og_0000                           NaN   

                                           summaries Note  
0  senior director of clinical services housing a...  NaN  
1  i believe that passions are meant to be shared...  NaN  
2  bill bryant is founder and chairman of bryant ...  NaN  
3  undertaking a trilingual masters degree in eur...  NaN  
4  career objective a role within marketing and n...  NaN  

 Technology Sheet Data 
   List of Technologies
0             .NETCore
1      [24]7 Live Chat
2        @hand SAPHRON
3                @RISK
4          @Value 2016

 output Data 
 Empty DataFrame
Columns: [id, summaries, Technology_Names]

In [79]:
# removing garbage words from tech sheet
r = pd.read_csv("../Remove.csv")

In [80]:
for i in range(0,len(r)):
    r["Remove"][i] = r["Remove"][i].lower()

In [81]:
remove_list = r["Remove"].tolist()
remove_list.append("ve")

In [82]:
len(tech)

29318

In [83]:
# sorting by id
summary = summary.sort_values(by=['id'], ascending=False)
summary.reset_index(inplace = True, drop = True)
summary.drop(columns = ["Note"], axis =1, inplace = True)

In [84]:
print(len(tech), len(summary))

29318 1001


In [85]:
# taking 500 entries from sheet
# summary = summary.iloc[0:100] 

In [86]:
tech.columns = ["title"]


In [87]:
# tech.columns = ["index", "title"]
# tech.drop(columns = ["index"], axis =1 , inplace = True)
tech.reset_index(inplace = True, drop = True)

In [88]:
tech_list = []
summary_list = []
id_list = []

In [89]:
summary["summaries"][0]

'senior director of clinical services housing assistance and hiv aids services with focus in clinical supervision trauma informed care trainer and consultant program development and leadership building behavior support specialist behavior therapist lcsw/ provide supervision to therapist obtaining credentials former management of autism circles of treatment support grant consultation in the area of intellectual disabilities and mental health needs expertise in the area and advocate of social and sexual awareness and needs for individuals participated and trained as a volunteer for the kentucky crisis response team volunteered for american red cross to debrief victims of katrina, flood and tropical storm disasters presentations in a variety of areas to assist clients and staff in behavioral and mental health needs former member of steering committee for trauma informed care and development of training, senior director of clinical services\nhousing programs with focus in clinical services

In [90]:
# converting string of list data that is being dumped from source here to list properly to iterate easily later on

def dump_format_change(df, col_name):
    
    for i in range(0, len(df)):
        print(i)
        if df[col_name][i] is not np.nan:
            demo = df[col_name][i]
            res = demo.strip('][')
            resl = []
            resl.append(res)
            df[col_name][i] = resl
        
        else:
            df[col_name][i] = '[]'
            demo = df[col_name][i]
            res = demo.strip('][')
            resl = []
            resl.append(res)
            df[col_name][i] = resl
    return df 

In [91]:
summary = dump_format_change(summary, "summaries")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [92]:
summary["exact_matched_patt_contextual"][33]

"['microsoft office', 'adobe']"

In [93]:

summary['exact_matched_patt_contextual'] = [ '[]' if x is np.NaN else x for x in summary['exact_matched_patt_contextual'] ]


In [94]:
# Converting dumped data into proper format i.e string into list

# Pass the matched technology column that we got
def convert_data_format(df,column_name):
    
    for i in range(0, len(df)):
        
        if df[column_name][i][0] == '[':
            res = ast.literal_eval(df[column_name][i])
            df[column_name][i] = res
            
        else:
            x = []
            x.insert(0, df[column_name][i])
            df[column_name][i] = x
            
    return df

In [95]:
summary = convert_data_format(summary, "exact_matched_patt_contextual")

In [96]:
summary["summaries"][0]

['senior director of clinical services housing assistance and hiv aids services with focus in clinical supervision trauma informed care trainer and consultant program development and leadership building behavior support specialist behavior therapist lcsw/ provide supervision to therapist obtaining credentials former management of autism circles of treatment support grant consultation in the area of intellectual disabilities and mental health needs expertise in the area and advocate of social and sexual awareness and needs for individuals participated and trained as a volunteer for the kentucky crisis response team volunteered for american red cross to debrief victims of katrina, flood and tropical storm disasters presentations in a variety of areas to assist clients and staff in behavioral and mental health needs former member of steering committee for trauma informed care and development of training, senior director of clinical services\nhousing programs with focus in clinical service

In [97]:
# trying joblib for paraller processing
#Import package
from joblib import Parallel, delayed
from joblib import Memory

In [98]:
backend = 'multiprocessing'

In [99]:
tech.head()

,title
0,.NETCore
1,[24]7 Live Chat
2,@hand SAPHRON
3,@RISK
4,@Value 2016


In [100]:
# tech=tech.iloc[:,1:2]

In [101]:
# tech.columns = ["title"]
import sys

In [102]:
tech['title'] = tech['title'].astype(str)
all_tech_words = list(tech['title'].str.lower())
summary["summaries_matching"]=summary["summaries"].astype(str)
all_strings = list(summary["summaries_matching"].str.lower())

In [103]:
# i=1
# tech_keys=[]
# spacy_keys = []
# for item in all_strings:
#     print("***","TEST No,",i,"***")
#     i=i+1
#     tech_row=[]
#     spacy_row = []
#     for k in all_tech_words:
#         if k in item and len(k)>2:
#             tech_row.append(k)
#             k=k.strip()
#             k.replace(" ","_")
#             spacy_row.append([k,item.index(k),item.index(k) + len(k),"ORG"])
        
#     tech_keys.append(tech_row)
#     spacy_keys.append(spacy_row)

In [104]:
i=1
tech_keys=[]
spacy_keys = []
for item in all_strings:
    print("***","TEST No,",i,"***")
    i=i+1
    tech_row=[]
    spacy_row = []
    for k in all_tech_words:
        if k in item and len(k)>2:
            tech_row.append(k)
            k=k.strip()
            k.replace(" ","_")
            k.replace(",","")
            spacy_row.append([k,item.index(k)-2,item.index(k) + len(k)-2,"ORG"])
        
    tech_keys.append(tech_row)
    spacy_keys.append(spacy_row)

*** TEST No, 1 ***
*** TEST No, 2 ***
*** TEST No, 3 ***
*** TEST No, 4 ***
*** TEST No, 5 ***
*** TEST No, 6 ***
*** TEST No, 7 ***
*** TEST No, 8 ***
*** TEST No, 9 ***
*** TEST No, 10 ***
*** TEST No, 11 ***
*** TEST No, 12 ***
*** TEST No, 13 ***
*** TEST No, 14 ***
*** TEST No, 15 ***
*** TEST No, 16 ***
*** TEST No, 17 ***
*** TEST No, 18 ***
*** TEST No, 19 ***
*** TEST No, 20 ***
*** TEST No, 21 ***
*** TEST No, 22 ***
*** TEST No, 23 ***
*** TEST No, 24 ***
*** TEST No, 25 ***
*** TEST No, 26 ***
*** TEST No, 27 ***
*** TEST No, 28 ***
*** TEST No, 29 ***
*** TEST No, 30 ***
*** TEST No, 31 ***
*** TEST No, 32 ***
*** TEST No, 33 ***
*** TEST No, 34 ***
*** TEST No, 35 ***
*** TEST No, 36 ***
*** TEST No, 37 ***
*** TEST No, 38 ***
*** TEST No, 39 ***
*** TEST No, 40 ***
*** TEST No, 41 ***
*** TEST No, 42 ***
*** TEST No, 43 ***
*** TEST No, 44 ***
*** TEST No, 45 ***
*** TEST No, 46 ***
*** TEST No, 47 ***
*** TEST No, 48 ***
*** TEST No, 49 ***
*** TEST No, 50 ***
*** TEST 

*** TEST No, 400 ***
*** TEST No, 401 ***
*** TEST No, 402 ***
*** TEST No, 403 ***
*** TEST No, 404 ***
*** TEST No, 405 ***
*** TEST No, 406 ***
*** TEST No, 407 ***
*** TEST No, 408 ***
*** TEST No, 409 ***
*** TEST No, 410 ***
*** TEST No, 411 ***
*** TEST No, 412 ***
*** TEST No, 413 ***
*** TEST No, 414 ***
*** TEST No, 415 ***
*** TEST No, 416 ***
*** TEST No, 417 ***
*** TEST No, 418 ***
*** TEST No, 419 ***
*** TEST No, 420 ***
*** TEST No, 421 ***
*** TEST No, 422 ***
*** TEST No, 423 ***
*** TEST No, 424 ***
*** TEST No, 425 ***
*** TEST No, 426 ***
*** TEST No, 427 ***
*** TEST No, 428 ***
*** TEST No, 429 ***
*** TEST No, 430 ***
*** TEST No, 431 ***
*** TEST No, 432 ***
*** TEST No, 433 ***
*** TEST No, 434 ***
*** TEST No, 435 ***
*** TEST No, 436 ***
*** TEST No, 437 ***
*** TEST No, 438 ***
*** TEST No, 439 ***
*** TEST No, 440 ***
*** TEST No, 441 ***
*** TEST No, 442 ***
*** TEST No, 443 ***
*** TEST No, 444 ***
*** TEST No, 445 ***
*** TEST No, 446 ***
*** TEST No, 

*** TEST No, 794 ***
*** TEST No, 795 ***
*** TEST No, 796 ***
*** TEST No, 797 ***
*** TEST No, 798 ***
*** TEST No, 799 ***
*** TEST No, 800 ***
*** TEST No, 801 ***
*** TEST No, 802 ***
*** TEST No, 803 ***
*** TEST No, 804 ***
*** TEST No, 805 ***
*** TEST No, 806 ***
*** TEST No, 807 ***
*** TEST No, 808 ***
*** TEST No, 809 ***
*** TEST No, 810 ***
*** TEST No, 811 ***
*** TEST No, 812 ***
*** TEST No, 813 ***
*** TEST No, 814 ***
*** TEST No, 815 ***
*** TEST No, 816 ***
*** TEST No, 817 ***
*** TEST No, 818 ***
*** TEST No, 819 ***
*** TEST No, 820 ***
*** TEST No, 821 ***
*** TEST No, 822 ***
*** TEST No, 823 ***
*** TEST No, 824 ***
*** TEST No, 825 ***
*** TEST No, 826 ***
*** TEST No, 827 ***
*** TEST No, 828 ***
*** TEST No, 829 ***
*** TEST No, 830 ***
*** TEST No, 831 ***
*** TEST No, 832 ***
*** TEST No, 833 ***
*** TEST No, 834 ***
*** TEST No, 835 ***
*** TEST No, 836 ***
*** TEST No, 837 ***
*** TEST No, 838 ***
*** TEST No, 839 ***
*** TEST No, 840 ***
*** TEST No, 

In [105]:
# summary = dump_format_change(summary, "summaries")

In [106]:
# spacy_keys

In [107]:
# tech_keys

In [108]:
summary['Tech_from_string_match']=tech_keys
summary.head()

,id,exact_matched_patt_contextual,summaries,summaries_matching,Tech_from_string_match
0,zsDSYc5UzpyXekOABZBfnQ_0000,[],[senior director of clinical services housing ...,['senior director of clinical services housing...,"[alis, and co, awareness, cat, circle, flo, in..."
1,zs1YNrrCeorFkGLEXSSp-A_0000,[],[i believe that passions are meant to be share...,['i believe that passions are meant to be shar...,"[alis, and co, ark, brain, cat, cien, construc..."
2,zowirq8ZhxhchApaRMhNSA_0000,[],[bill bryant is founder and chairman of bryant...,"[""bill bryant is founder and chairman of bryan...","[air, candid, cat, chai, found, foundation, in..."
3,zoQVkPCfXB9n3AaPGHHwzg_0000,[],[undertaking a trilingual masters degree in eu...,['undertaking a trilingual masters degree in e...,"[alis, and co, ark, cat, emplo, enth, ics, ion..."
4,zmIr2glBZ3Ef8CSS0jw1og_0000,[],[career objective a role within marketing and ...,"[""career objective a role within marketing and...","[ami, and co, ark, cat, combin, front, ion, it..."


In [109]:
# summary["exact_matched_patt_contextual"][11]

In [110]:
summary['spacy_format_v1']=spacy_keys
summary.head()

,id,exact_matched_patt_contextual,summaries,summaries_matching,Tech_from_string_match,spacy_format_v1
0,zsDSYc5UzpyXekOABZBfnQ_0000,[],[senior director of clinical services housing ...,['senior director of clinical services housing...,"[alis, and co, awareness, cat, circle, flo, in...","[[alis, 223, 227, ORG], [and co, 142, 148, ORG..."
1,zs1YNrrCeorFkGLEXSSp-A_0000,[],[i believe that passions are meant to be share...,['i believe that passions are meant to be shar...,"[alis, and co, ark, brain, cat, cien, construc...","[[alis, 1135, 1139, ORG], [and co, 358, 364, O..."
2,zowirq8ZhxhchApaRMhNSA_0000,[],[bill bryant is founder and chairman of bryant...,"[""bill bryant is founder and chairman of bryan...","[air, candid, cat, chai, found, foundation, in...","[[air, 29, 32, ORG], [candid, 695, 701, ORG], ..."
3,zoQVkPCfXB9n3AaPGHHwzg_0000,[],[undertaking a trilingual masters degree in eu...,['undertaking a trilingual masters degree in e...,"[alis, and co, ark, cat, emplo, enth, ics, ion...","[[alis, 94, 98, ORG], [and co, 434, 440, ORG],..."
4,zmIr2glBZ3Ef8CSS0jw1og_0000,[],[career objective a role within marketing and ...,"[""career objective a role within marketing and...","[ami, and co, ark, cat, combin, front, ion, it...","[[ami, 602, 605, ORG], [and co, 359, 365, ORG]..."


In [111]:
summary["summaries"][0]

['senior director of clinical services housing assistance and hiv aids services with focus in clinical supervision trauma informed care trainer and consultant program development and leadership building behavior support specialist behavior therapist lcsw/ provide supervision to therapist obtaining credentials former management of autism circles of treatment support grant consultation in the area of intellectual disabilities and mental health needs expertise in the area and advocate of social and sexual awareness and needs for individuals participated and trained as a volunteer for the kentucky crisis response team volunteered for american red cross to debrief victims of katrina, flood and tropical storm disasters presentations in a variety of areas to assist clients and staff in behavioral and mental health needs former member of steering committee for trauma informed care and development of training, senior director of clinical services\nhousing programs with focus in clinical service

In [112]:
df = summary

In [113]:
df["summaries"][0]

['senior director of clinical services housing assistance and hiv aids services with focus in clinical supervision trauma informed care trainer and consultant program development and leadership building behavior support specialist behavior therapist lcsw/ provide supervision to therapist obtaining credentials former management of autism circles of treatment support grant consultation in the area of intellectual disabilities and mental health needs expertise in the area and advocate of social and sexual awareness and needs for individuals participated and trained as a volunteer for the kentucky crisis response team volunteered for american red cross to debrief victims of katrina, flood and tropical storm disasters presentations in a variety of areas to assist clients and staff in behavioral and mental health needs former member of steering committee for trauma informed care and development of training, senior director of clinical services\nhousing programs with focus in clinical service

In [114]:
# def final_run_string(summary_copy, tech):
    
#     tech_list = []
#     summary_list = []
#     id_list = []
    
#     start = time.time()

#     for i in range(0, len(summary_copy)):
#         print(i)
#         string = summary_copy.loc[i, "summaries"][0]
#         string = string.lower()

#         for j in range(0, len(tech)):
#             pat = tech.loc[j,"title"]
#             pat = pat.lower()


#             initial_index = KMP_String(pat, string)
#             if len(initial_index)!=0:

#                 tech_list.append(pat)    
#                 summary_list.append(string)
#                 id_list.append(summary_copy.loc[i,"id"])

#     end = time.time() - start
#     print("\n Time Taken \n", end)
#     return tech_list, summary_list, id_list

In [115]:
# tech_list, summary_list, id_list = final_run_string(summary, tech)

In [116]:
# tech_list_copy = tech_list
# id_list_copy = id_list
# summary_list_copy = summary_list

In [117]:
# def getDuplicatesWithInfo(listOfElems):
#     ''' Get duplicate element in a list along with thier indices in list
#      and frequency count'''
#     dictOfElems = dict()
#     index = 0
#     # Iterate over each element in list and keep track of index
#     for elem in listOfElems:
#         # If element exists in dict then keep its index in list & increment its frequency
#         if elem in dictOfElems:
#             dictOfElems[elem][0] += 1
#             dictOfElems[elem][1].append(index)
#         else:
#             # Add a new entry in dictionary 
#             dictOfElems[elem] = [1, [index]]
#         index += 1    
 
#     dictOfElems = { key:value for key, value in dictOfElems.items() if value[0] > 1}
#     return dictOfElems

In [118]:
# # List of strings
# listOfElems = id_list
# dictOfElems = getDuplicatesWithInfo(listOfElems)

In [119]:
# df = pd.DataFrame(list(zip(id_list, summary_list, tech_list)), 
#                columns =['id', 'summaries', "Tech_from_string_match"])

In [120]:
# for key, value in dictOfElems.items():
# #     print("******************")
# #     print("\n ID is..", key )
#     a = df["Tech_from_string_match"][value[1][0]] #this needs to be changed
#     b = []
#     b.insert(0,a)
# #     print("List at first index", b)
#     for i in range(0, len(value[1])-1):
#         b.append(df["Tech_from_string_match"][value[1][i+1]])
#         df["Tech_from_string_match"][value[1][0]] = b
        
#     for i in range(0, len(value[1])-1):
#         df = df.drop(index = [value[1][i+1]])
# #         print('Element = ', key , ' :: Repeated Count = ', value[0] , ' :: Index Positions =  ', value[1])

In [121]:
df.reset_index(inplace = True, drop = True)

In [122]:
# Applying Regex to filter out Invalid Words

def get_exact_match(txt, tech):
    try:
        patt = '|'.join(['\\b'+elem+'\\b' for elem in tech])
        matched_patt = re.findall(patt,txt)
    except Exception as e:
        print(e)
        matched_patt = []
    return matched_patt

In [123]:
df['exact_matched_patt'] = df.apply(lambda x: get_exact_match(x["summaries"][0], x["Tech_from_string_match"]), axis = 1)

multiple repeat at position 20
multiple repeat at position 50
multiple repeat at position 20
multiple repeat at position 47
multiple repeat at position 40
multiple repeat at position 102
multiple repeat at position 55
multiple repeat at position 35
multiple repeat at position 71
multiple repeat at position 67
multiple repeat at position 20
multiple repeat at position 51
multiple repeat at position 78
multiple repeat at position 21
multiple repeat at position 20


In [124]:
df.head()

,id,exact_matched_patt_contextual,summaries,summaries_matching,Tech_from_string_match,spacy_format_v1,exact_matched_patt
0,zsDSYc5UzpyXekOABZBfnQ_0000,[],[senior director of clinical services housing ...,['senior director of clinical services housing...,"[alis, and co, awareness, cat, circle, flo, in...","[[alis, 223, 227, ORG], [and co, 142, 148, ORG...","[awareness, tropical, storm, awareness, tropic..."
1,zs1YNrrCeorFkGLEXSSp-A_0000,[],[i believe that passions are meant to be share...,['i believe that passions are meant to be shar...,"[alis, and co, ark, brain, cat, cien, construc...","[[alis, 1135, 1139, ORG], [and co, 358, 364, O...","[storyteller, brain, brain, grow]"
2,zowirq8ZhxhchApaRMhNSA_0000,[],[bill bryant is founder and chairman of bryant...,"[""bill bryant is founder and chairman of bryan...","[air, candid, cat, chai, found, foundation, in...","[[air, 29, 32, ORG], [candid, 695, 701, ORG], ...",[foundation]
3,zoQVkPCfXB9n3AaPGHHwzg_0000,[],[undertaking a trilingual masters degree in eu...,['undertaking a trilingual masters degree in e...,"[alis, and co, ark, cat, emplo, enth, ics, ion...","[[alis, 94, 98, ORG], [and co, 434, 440, ORG],...",[]
4,zmIr2glBZ3Ef8CSS0jw1og_0000,[],[career objective a role within marketing and ...,"[""career objective a role within marketing and...","[ami, and co, ark, cat, combin, front, ion, it...","[[ami, 602, 605, ORG], [and co, 359, 365, ORG]...","[speak, front]"


In [125]:
df["spacy_format_v1"][0]

[['alis', 223, 227, 'ORG'],
 ['and co', 142, 148, 'ORG'],
 ['awareness', 506, 515, 'ORG'],
 ['cat', 480, 483, 'ORG'],
 ['circle', 337, 343, 'ORG'],
 ['flo', 686, 689, 'ORG'],
 ['inform', 120, 126, 'ORG'],
 ['intel', 400, 405, 'ORG'],
 ['intellect', 400, 409, 'ORG'],
 ['ion', 109, 112, 'ORG'],
 ['nex', 1346, 1349, 'ORG'],
 ['ning', 292, 296, 'ORG'],
 ['ora', 794, 797, 'ORG'],
 ['rain', 135, 139, 'ORG'],
 ['soci', 488, 492, 'ORG'],
 ['staf', 779, 783, 'ORG'],
 ['steer', 840, 845, 'ORG'],
 ['storm', 705, 710, 'ORG'],
 ['tia', 303, 306, 'ORG'],
 ['tor', 12, 15, 'ORG'],
 ['tropical', 696, 704, 'ORG'],
 ['vision', 106, 112, 'ORG'],
 ['vocate', 478, 484, 'ORG']]

In [126]:
# Applying Regex to filter out Invalid Words

def get_exact_match_2(txt, tech):
    try:
        patt = '|'.join(['\\s'+elem+'\\s' for elem in tech])
        matched_patt = re.findall(patt,txt[0])
    except Exception as e:
        print(e)
        matched_patt = []
    return matched_patt

In [127]:
df['matched_NER_final'] = df.apply(lambda x: get_exact_match_2(x["summaries"][0], x["exact_matched_patt"]), axis = 1)

In [128]:
from collections import OrderedDict 

In [129]:
for ind in df.index:
    res = list(OrderedDict.fromkeys(df["exact_matched_patt"][ind]))
    df["exact_matched_patt"][ind] = res

In [130]:
for i in range(0, len(df)):
    for word in list(df["exact_matched_patt"][i]):
        if word in remove_list:
            df["exact_matched_patt"][i].remove(word)

In [131]:
df.head()

,id,exact_matched_patt_contextual,summaries,summaries_matching,Tech_from_string_match,spacy_format_v1,exact_matched_patt,matched_NER_final
0,zsDSYc5UzpyXekOABZBfnQ_0000,[],[senior director of clinical services housing ...,['senior director of clinical services housing...,"[alis, and co, awareness, cat, circle, flo, in...","[[alis, 223, 227, ORG], [and co, 142, 148, ORG...","[awareness, tropical, storm]",[]
1,zs1YNrrCeorFkGLEXSSp-A_0000,[],[i believe that passions are meant to be share...,['i believe that passions are meant to be shar...,"[alis, and co, ark, brain, cat, cien, construc...","[[alis, 1135, 1139, ORG], [and co, 358, 364, O...","[storyteller, brain, grow]",[]
2,zowirq8ZhxhchApaRMhNSA_0000,[],[bill bryant is founder and chairman of bryant...,"[""bill bryant is founder and chairman of bryan...","[air, candid, cat, chai, found, foundation, in...","[[air, 29, 32, ORG], [candid, 695, 701, ORG], ...",[foundation],[]
3,zoQVkPCfXB9n3AaPGHHwzg_0000,[],[undertaking a trilingual masters degree in eu...,['undertaking a trilingual masters degree in e...,"[alis, and co, ark, cat, emplo, enth, ics, ion...","[[alis, 94, 98, ORG], [and co, 434, 440, ORG],...",[],"[, ]"
4,zmIr2glBZ3Ef8CSS0jw1og_0000,[],[career objective a role within marketing and ...,"[""career objective a role within marketing and...","[ami, and co, ark, cat, combin, front, ion, it...","[[ami, 602, 605, ORG], [and co, 359, 365, ORG]...","[speak, front]",[]


In [132]:
df["spacy_format_v1"][0]

[['alis', 223, 227, 'ORG'],
 ['and co', 142, 148, 'ORG'],
 ['awareness', 506, 515, 'ORG'],
 ['cat', 480, 483, 'ORG'],
 ['circle', 337, 343, 'ORG'],
 ['flo', 686, 689, 'ORG'],
 ['inform', 120, 126, 'ORG'],
 ['intel', 400, 405, 'ORG'],
 ['intellect', 400, 409, 'ORG'],
 ['ion', 109, 112, 'ORG'],
 ['nex', 1346, 1349, 'ORG'],
 ['ning', 292, 296, 'ORG'],
 ['ora', 794, 797, 'ORG'],
 ['rain', 135, 139, 'ORG'],
 ['soci', 488, 492, 'ORG'],
 ['staf', 779, 783, 'ORG'],
 ['steer', 840, 845, 'ORG'],
 ['storm', 705, 710, 'ORG'],
 ['tia', 303, 306, 'ORG'],
 ['tor', 12, 15, 'ORG'],
 ['tropical', 696, 704, 'ORG'],
 ['vision', 106, 112, 'ORG'],
 ['vocate', 478, 484, 'ORG']]

In [133]:
# df["exact_matched_patt_contextual"].fillna('[]')
df["exact_matched_patt"][3]

[]

In [134]:
df["exact_matched_patt_contextual"][33]

['microsoft office', 'adobe']

In [135]:
# df['exact_matched_patt_contextual'] = [ [] if x is np.NaN else x for x in df['exact_matched_patt_contextual'] ]


In [136]:
df

,id,exact_matched_patt_contextual,summaries,summaries_matching,Tech_from_string_match,spacy_format_v1,exact_matched_patt,matched_NER_final
0,zsDSYc5UzpyXekOABZBfnQ_0000,[],[senior director of clinical services housing ...,['senior director of clinical services housing...,"[alis, and co, awareness, cat, circle, flo, in...","[[alis, 223, 227, ORG], [and co, 142, 148, ORG...","[awareness, tropical, storm]",[]
1,zs1YNrrCeorFkGLEXSSp-A_0000,[],[i believe that passions are meant to be share...,['i believe that passions are meant to be shar...,"[alis, and co, ark, brain, cat, cien, construc...","[[alis, 1135, 1139, ORG], [and co, 358, 364, O...","[storyteller, brain, grow]",[]
2,zowirq8ZhxhchApaRMhNSA_0000,[],[bill bryant is founder and chairman of bryant...,"[""bill bryant is founder and chairman of bryan...","[air, candid, cat, chai, found, foundation, in...","[[air, 29, 32, ORG], [candid, 695, 701, ORG], ...",[foundation],[]
3,zoQVkPCfXB9n3AaPGHHwzg_0000,[],[undertaking a trilingual masters degree in eu...,['undertaking a trilingual masters degree in e...,"[alis, and co, ark, cat, emplo, enth, ics, ion...","[[alis, 94, 98, ORG], [and co, 434, 440, ORG],...",[],"[, ]"
4,zmIr2glBZ3Ef8CSS0jw1og_0000,[],[career objective a role within marketing and ...,"[""career objective a role within marketing and...","[ami, and co, ark, cat, combin, front, ion, it...","[[ami, 602, 605, ORG], [and co, 359, 365, ORG]...","[speak, front]",[]
...,...,...,...,...,...,...,...,...
996,-EQeYEq-03cWVCkfnsUgow_0000,[],"[eric is an experienced risk, insurance, and p...","['eric is an experienced risk, insurance, and ...","[ami, ark, ava, big picture, cat, communicator...","[[ami, 454, 457, ORG], [ark, 671, 674, ORG], [...","[communicator, big picture]",[]
997,-EJPzKCOhrcLUKsIiC35Ow_0000,[],[team manager with 8 years of experience enabl...,['team manager with 8 years of experience enab...,"[ace, ami, cat, chai, devi, directly, ecos, em...","[[ace, 318, 321, ORG], [ami, 344, 347, ORG], [...","[stay, lob, later]",[]
998,-BSYFSH4GWqbTH-JhxVExA_0000,[],[i am currently a junior at cal poly san luis ...,"[""i am currently a junior at cal poly san luis...","[ace, actor, ark, astro, cat, continu, esign, ...","[[ace, 1719, 1722, ORG], [actor, 2430, 2435, O...","[gain, astro, honestly]",[]
999,-4ldAQ53se0hp9GU5FgnIQ_0000,[],[an extremely driven professional within the b...,['an extremely driven professional within the ...,"[ace, and co, assess, attend, batch, cat, cien...","[[ace, 192, 195, ORG], [and co, 765, 771, ORG]...","[batch, attend, daily]",[]


In [137]:
# remove invalid entries from spacy format column

def remove_invalid(final_tech, spacy_format):
    
    for i in list(spacy_format):
        if i[0] not in final_tech:
            spacy_format.remove(i)

    return spacy_format          

In [138]:
df["spacy_format_manual_tag"] = df.apply(lambda x: remove_invalid(x["exact_matched_patt_contextual"], x["spacy_format_v1"]), axis = 1)

In [139]:
# df.to_csv("lookup_data.csv", index = False)

# Commenting removing technology name function for now

In [140]:
df.head()

,id,exact_matched_patt_contextual,summaries,summaries_matching,Tech_from_string_match,spacy_format_v1,exact_matched_patt,matched_NER_final,spacy_format_manual_tag
0,zsDSYc5UzpyXekOABZBfnQ_0000,[],[senior director of clinical services housing ...,['senior director of clinical services housing...,"[alis, and co, awareness, cat, circle, flo, in...",[],"[awareness, tropical, storm]",[],[]
1,zs1YNrrCeorFkGLEXSSp-A_0000,[],[i believe that passions are meant to be share...,['i believe that passions are meant to be shar...,"[alis, and co, ark, brain, cat, cien, construc...",[],"[storyteller, brain, grow]",[],[]
2,zowirq8ZhxhchApaRMhNSA_0000,[],[bill bryant is founder and chairman of bryant...,"[""bill bryant is founder and chairman of bryan...","[air, candid, cat, chai, found, foundation, in...",[],[foundation],[],[]
3,zoQVkPCfXB9n3AaPGHHwzg_0000,[],[undertaking a trilingual masters degree in eu...,['undertaking a trilingual masters degree in e...,"[alis, and co, ark, cat, emplo, enth, ics, ion...",[],[],"[, ]",[]
4,zmIr2glBZ3Ef8CSS0jw1og_0000,[],[career objective a role within marketing and ...,"[""career objective a role within marketing and...","[ami, and co, ark, cat, combin, front, ion, it...",[],"[speak, front]",[],[]


In [141]:
len(df["exact_matched_patt"][0])

3

In [142]:
index_drop = []
for i in range(0, len(df)):
    print(i)
    if len(df["spacy_format_manual_tag"][i])==0:
        index_drop.append(i)
df = df.drop(labels = index_drop, axis = 0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [143]:
len(df)

288

In [144]:
df

,id,exact_matched_patt_contextual,summaries,summaries_matching,Tech_from_string_match,spacy_format_v1,exact_matched_patt,matched_NER_final,spacy_format_manual_tag
6,zlBvzxx47BewWlO67-zK3Q_0000,[republic],[i am a 19 year old entrepreneur and manager o...,"[""i am a 19 year old entrepreneur and manager ...","[ada, and co, construct, countr, ebs, found, g...","[[republic, 196, 204, ORG]]","[republic, turn, grow]",[],"[[republic, 196, 204, ORG]]"
7,zekuTL0ONYTX2XawUBa-9A_0000,[dell],[i am a self motivated professional with 13+ y...,['i am a self motivated professional with 13+ ...,"[ada, ark, cat, d custom, dell, emplo, ento, g...","[[dell, 2431, 2435, ORG]]","[grow, dell]",[],"[[dell, 2431, 2435, ORG]]"
8,zaByWGsar5hVdBWLEYykcg_0000,"[wordpress, joomla, drupal]",[as owner and director of projects at metro in...,['as owner and director of projects at metro i...,"[accurate, amo, and co, ark, blis, cat, ddi, d...","[[drupal, 940, 946, ORG], [joomla, 921, 927, O...","[promote, accurate, medium, portal, joomla, wo...",[],"[[drupal, 940, 946, ORG], [joomla, 921, 927, O..."
9,zPnnWGWHb2gSYnd98GKaag_0000,[unity],[15+ years within the recruiting industry comb...,['15+ years within the recruiting industry com...,"[candid, capabiliti, cat, combin, ento, forge,...","[[unity, 3286, 3291, ORG]]","[here, guru, unicorn, purple, hunter, snap, un...",[],"[[unity, 3286, 3291, ORG]]"
12,z5LcE0BcZevf5WmwL8nunQ_0000,"[microsoft office, juniper, workday, python, l...","[bachelor of science (b.s) degree in "" telecom...","['bachelor of science (b.s) degree in "" teleco...","[alis, ava, cat, cien, electron, eploy, format...","[[juniper, 955, 962, ORG], [linux, 992, 997, O...","[route, switch, link, juniper, linux, python, ...",[],"[[juniper, 955, 962, ORG], [linux, 992, 997, O..."
...,...,...,...,...,...,...,...,...,...
984,-nbbpInS-iD0thomdMs6BQ_0000,[zoom],[i am a self-motivated sales & marketing profe...,"[""i am a self-motivated sales & marketing prof...","[air, ami, ark, cat, construct, continu, ddi, ...","[[zoom, 689, 693, ORG]]","[rapport, zoom, stay, promote, wing, flight]",[],"[[zoom, 689, 693, ORG]]"
985,-jsY6ula0lsaOQ5rBHxHLg_0000,"[invision, pixate, xcode, html]",[www.lisawilkins.com specialties: visual/ui/ux...,"[""www.lisawilkins.com specialties: visual/ui/u...","[accept, aps, ava, cod, connect, construct, de...","[[html, 382, 386, ORG], [invision, 341, 349, O...","[invisible, invision, pixate, soon, html, unif...",[],"[[html, 382, 386, ORG], [invision, 341, 349, O..."
986,-iRef22R0w52UJT280zoxQ_0000,"[bootstrap, sitecore, asp.net, jquery]",[senior software engineer with 6+ years in ana...,['senior software engineer with 6+ years in an...,"[ami, amo, apt, asp.net, bootstrap, cat, esign...","[[asp.net, 288, 295, ORG], [bootstrap, 395, 40...","[asp.net, jquery, bootstrap, less, proved, sol...",[],"[[asp.net, 288, 295, ORG], [bootstrap, 395, 40..."
990,-bBI1d6Emr0wtwLVEvrssw_0000,[groupe],"[managing director, ceo, strategic business un...","['managing director, ceo, strategic business u...","[ark, construct, electron, esign, groupe, grow...","[[groupe, 1678, 1684, ORG]]","[sigma, groupe]",[],"[[groupe, 1678, 1684, ORG]]"


In [145]:
df.reset_index(inplace = True, drop = True)

In [146]:
df.to_csv("lookup_data.csv", index = False)

In [147]:
def remove_name(spacy_format_final):
    for i in range(0, len(list(spacy_format_final))):
        name = spacy_format_final[i][0]
        spacy_format_final[i].remove(name)
        
    return spacy_format_final

In [148]:
df["spacy_format_manual_tag"] = df.apply(lambda x: remove_name(x["spacy_format_manual_tag"]), axis = 1)

In [149]:
# df = df.loc[(df["exact_matched_patt"]) == '[]']
df.head()

,id,exact_matched_patt_contextual,summaries,summaries_matching,Tech_from_string_match,spacy_format_v1,exact_matched_patt,matched_NER_final,spacy_format_manual_tag
0,zlBvzxx47BewWlO67-zK3Q_0000,[republic],[i am a 19 year old entrepreneur and manager o...,"[""i am a 19 year old entrepreneur and manager ...","[ada, and co, construct, countr, ebs, found, g...","[[196, 204, ORG]]","[republic, turn, grow]",[],"[[196, 204, ORG]]"
1,zekuTL0ONYTX2XawUBa-9A_0000,[dell],[i am a self motivated professional with 13+ y...,['i am a self motivated professional with 13+ ...,"[ada, ark, cat, d custom, dell, emplo, ento, g...","[[2431, 2435, ORG]]","[grow, dell]",[],"[[2431, 2435, ORG]]"
2,zaByWGsar5hVdBWLEYykcg_0000,"[wordpress, joomla, drupal]",[as owner and director of projects at metro in...,['as owner and director of projects at metro i...,"[accurate, amo, and co, ark, blis, cat, ddi, d...","[[940, 946, ORG], [921, 927, ORG], [929, 938, ...","[promote, accurate, medium, portal, joomla, wo...",[],"[[940, 946, ORG], [921, 927, ORG], [929, 938, ..."
3,zPnnWGWHb2gSYnd98GKaag_0000,[unity],[15+ years within the recruiting industry comb...,['15+ years within the recruiting industry com...,"[candid, capabiliti, cat, combin, ento, forge,...","[[3286, 3291, ORG]]","[here, guru, unicorn, purple, hunter, snap, un...",[],"[[3286, 3291, ORG]]"
4,z5LcE0BcZevf5WmwL8nunQ_0000,"[microsoft office, juniper, workday, python, l...","[bachelor of science (b.s) degree in "" telecom...","['bachelor of science (b.s) degree in "" teleco...","[alis, ava, cat, cien, electron, eploy, format...","[[955, 962, ORG], [992, 997, ORG], [2336, 2352...","[route, switch, link, juniper, linux, python, ...",[],"[[955, 962, ORG], [992, 997, ORG], [2336, 2352..."


In [150]:
len(df)

288

In [151]:
def format_arrange(summaries, spacy_format_final):
    
    d = {}
    for i in spacy_format_final:
        i = tuple(i)
        d.setdefault("entities", [])
        d["entities"].append(i)    
    summary_tuple = (summaries[0], d)

    return summary_tuple

In [152]:
df["spacy_format_manual_tag"] = df.apply(lambda x: format_arrange(x["summaries"], x["spacy_format_manual_tag"]), axis = 1)

In [153]:
df.head()

,id,exact_matched_patt_contextual,summaries,summaries_matching,Tech_from_string_match,spacy_format_v1,exact_matched_patt,matched_NER_final,spacy_format_manual_tag
0,zlBvzxx47BewWlO67-zK3Q_0000,[republic],[i am a 19 year old entrepreneur and manager o...,"[""i am a 19 year old entrepreneur and manager ...","[ada, and co, construct, countr, ebs, found, g...","[[196, 204, ORG]]","[republic, turn, grow]",[],(i am a 19 year old entrepreneur and manager o...
1,zekuTL0ONYTX2XawUBa-9A_0000,[dell],[i am a self motivated professional with 13+ y...,['i am a self motivated professional with 13+ ...,"[ada, ark, cat, d custom, dell, emplo, ento, g...","[[2431, 2435, ORG]]","[grow, dell]",[],(i am a self motivated professional with 13+ y...
2,zaByWGsar5hVdBWLEYykcg_0000,"[wordpress, joomla, drupal]",[as owner and director of projects at metro in...,['as owner and director of projects at metro i...,"[accurate, amo, and co, ark, blis, cat, ddi, d...","[[940, 946, ORG], [921, 927, ORG], [929, 938, ...","[promote, accurate, medium, portal, joomla, wo...",[],(as owner and director of projects at metro in...
3,zPnnWGWHb2gSYnd98GKaag_0000,[unity],[15+ years within the recruiting industry comb...,['15+ years within the recruiting industry com...,"[candid, capabiliti, cat, combin, ento, forge,...","[[3286, 3291, ORG]]","[here, guru, unicorn, purple, hunter, snap, un...",[],(15+ years within the recruiting industry comb...
4,z5LcE0BcZevf5WmwL8nunQ_0000,"[microsoft office, juniper, workday, python, l...","[bachelor of science (b.s) degree in "" telecom...","['bachelor of science (b.s) degree in "" teleco...","[alis, ava, cat, cien, electron, eploy, format...","[[955, 962, ORG], [992, 997, ORG], [2336, 2352...","[route, switch, link, juniper, linux, python, ...",[],"(bachelor of science (b.s) degree in "" telecom..."


In [154]:
df["spacy_format_manual_tag"][4]

('bachelor of science (b.s) degree in " telecommunications networks and software " program at politehnica university - electronics ,telecommunications and information technology faculty master of science ( m.s.) degree in " management of networks and services " program at politehnica university - electronics ,telecommunications and information technology faculty routing and switching skills cisco ccna ( 640-802 certification ) cisco ccnp route ( 642-902 certification : implementing cisco ip routing ) cisco ccnp switch ( 642-813 certification: implementing cisco ip switched networks ) cisco ccnp tshoot ( 642-832 certification: troubleshooting and maintaining cisco ip networks ) experience in troubleshooting and deploying networks using major l2/l3 protocols and platforms: - routing protocols (rip, ospf, eigrp,is-is,bgp) - data link protocols (stp, rstp,mstp) - first hop redundancy protocols: hsrp, vrrp, glbp - mpls fundamentals - cisco, nexus, juniper, alcatel sr platforms - unix/linux 

In [155]:
df.head()

,id,exact_matched_patt_contextual,summaries,summaries_matching,Tech_from_string_match,spacy_format_v1,exact_matched_patt,matched_NER_final,spacy_format_manual_tag
0,zlBvzxx47BewWlO67-zK3Q_0000,[republic],[i am a 19 year old entrepreneur and manager o...,"[""i am a 19 year old entrepreneur and manager ...","[ada, and co, construct, countr, ebs, found, g...","[[196, 204, ORG]]","[republic, turn, grow]",[],(i am a 19 year old entrepreneur and manager o...
1,zekuTL0ONYTX2XawUBa-9A_0000,[dell],[i am a self motivated professional with 13+ y...,['i am a self motivated professional with 13+ ...,"[ada, ark, cat, d custom, dell, emplo, ento, g...","[[2431, 2435, ORG]]","[grow, dell]",[],(i am a self motivated professional with 13+ y...
2,zaByWGsar5hVdBWLEYykcg_0000,"[wordpress, joomla, drupal]",[as owner and director of projects at metro in...,['as owner and director of projects at metro i...,"[accurate, amo, and co, ark, blis, cat, ddi, d...","[[940, 946, ORG], [921, 927, ORG], [929, 938, ...","[promote, accurate, medium, portal, joomla, wo...",[],(as owner and director of projects at metro in...
3,zPnnWGWHb2gSYnd98GKaag_0000,[unity],[15+ years within the recruiting industry comb...,['15+ years within the recruiting industry com...,"[candid, capabiliti, cat, combin, ento, forge,...","[[3286, 3291, ORG]]","[here, guru, unicorn, purple, hunter, snap, un...",[],(15+ years within the recruiting industry comb...
4,z5LcE0BcZevf5WmwL8nunQ_0000,"[microsoft office, juniper, workday, python, l...","[bachelor of science (b.s) degree in "" telecom...","['bachelor of science (b.s) degree in "" teleco...","[alis, ava, cat, cien, electron, eploy, format...","[[955, 962, ORG], [992, 997, ORG], [2336, 2352...","[route, switch, link, juniper, linux, python, ...",[],"(bachelor of science (b.s) degree in "" telecom..."


In [156]:
TRAIN_DATA = []

for i in range(0, len(df)):
    
    TRAIN_DATA.append(df["spacy_format_manual_tag"][i])
    


In [158]:
# TRAIN_DATA

# Checking training lg model STARTS HERE...

In [159]:
# import random
# import spacy
# from spacy.training import Example


In [160]:
# nlp = spacy.load('en_core_web_lg')  # load existing spaCy model
# ner = nlp.get_pipe('ner')
# ner.add_label("ORG")
# print(ner.move_names) # Here I see, that the new label was added
# optimizer = nlp.create_optimizer()
# # get names of other pipes to disable them during training
# other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
# with nlp.disable_pipes(*other_pipes):  # only train NER
#     for itn in range(2):
#         random.shuffle(TRAIN_DATA)
#         losses = {}
#         for text, annotations in TRAIN_DATA:
#             doc = nlp.make_doc(text)
#             example = Example.from_dict(doc, annotations)
            
#             nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
#         print(losses)
# # test the trained model # add some dummy sentences with many NERs

In [161]:
# # test the trained model # add some dummy sentences with many NERs

# test_text = 'Do you like horses?'
# doc = nlp(test_text)
# print("Entities in '%s'" % test_text)
# for ent in doc.ents:
#     print(ent.label_, " -- ", ent.text)

# Checking training lg model ENDS HERE...

In [162]:
len(TRAIN_DATA)

288

In [163]:
TRAINING_DATA = TRAIN_DATA[:200]
TEST_DATA = TRAIN_DATA[200:]

In [164]:
print(len(TRAINING_DATA), len(TEST_DATA))

200 88


In [165]:
def load_data(file):
    with open(file, "r", encoding = "utf-8") as f:
        data = json.load(f)
    return (data)

In [166]:
def save_data(file, data):
    with open(file, "w", encoding = "utf-8") as f:
        json.dump(data, f, indent = 4)

In [167]:
# df.to_csv("KMP_string_Match_newer_1000.csv", index = False)

In [168]:
save_data("spacy_dataset/training_data_spacy.json", TRAINING_DATA)
save_data("spacy_dataset/validation_data_spacy.json", TEST_DATA)

In [169]:
train = load_data("spacy_dataset/training_data_spacy.json")
valid = load_data("spacy_dataset/validation_data_spacy.json")

In [170]:
train[12]

["highly organized, creative, want to be involved in an inspiring project where i can utilize skill and creatively involved with system that effectively contributes to the growth of the project and putting a smile on my customers faces.\n\nmy work experience has provided me with the skills to take charge in all facets of planning and coordinating events, activities, tasks, contract negotiation and also use my creative skills to design any printed materials such as brochures, flyers and posters, ..., etc as per the event requirements. \n\non the other hand, my administrative expertise allowed me to build on my skills in managing multiple task and being adaptable to unpredictable scenarios and challenges. i also possess considerable knowledge in working with computer software programs such as microsoft office and adobe.\n\ni enjoy a demanding and variable work environments and am open to work on new and challenging tasks to enhance my knowledge and career., i have many years of experienc

# Training_conversion

In [171]:
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

In [172]:
# nlp = spacy.blank("en") # load a new spacy model
# db = DocBin() # create a DocBin object

In [173]:
# for text, annot in tqdm(TRAIN_DATA): # data in previous format
#     doc = nlp.make_doc(text) # create doc object from text
#     ents = []
#     try:
#         for start, end, label in annot["entities"]: # add character indexes
# #             print("\n start \n", start, "\n end \n", end, "\n label \n", label)
#             span = doc.char_span(start, end, label=label, alignment_mode="contract")
# #             print("\n span \n", span)
#             if span is None:
#                 print("Skipping entity")
#             else:
#                 ents.append(span)
#         doc.ents = ents # label the text with the ents
#         db.add(doc)
#     except Exception as e:
#         pass

In [174]:
# for i in train:
#     print(i)
import sys

# All is correct but spans are generated in a wrong way

In [175]:
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object
check_ents = []
for text, annot in tqdm(train):
#     print("\nannot\n", annot)# data in previous format
    doc = nlp.make_doc(text) # create doc object from text
#     print("\ndoc\n", doc)
    ents = []
    try:
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="strict")
#             print("\nspan\n",span,"\nannot\n", annot)
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        check_ents.append(ents)
#         doc.ents = [e for e in doc.ents if not e.text.isspace()]
        db.add(doc)
    except Exception as e:
        pass
# print(check_ents)
db.to_disk("/Users/shubhamsunwalka/Desktop/Slintel_2/training_spacy/train_2.spacy") # save the docbin object

 10%|█         | 21/200 [00:00<00:01, 107.47it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 27%|██▋       | 54/200 [00:00<00:01, 142.39it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 51%|█████     | 102/200 [00:00<00:00, 153.33it/s]

...

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 86%|████████▋ | 173/200 [00:01<00:00, 152.57it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity

100%|██████████| 200/200 [00:01<00:00, 146.12it/s]



Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [176]:
import spacy
from spacy.training import Example
from spacy.tokens import DocBin

In [177]:
!pwd

/Users/shubhamsunwalka/Desktop/Slintel_2/training_spacy


In [178]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [180]:
!spacy debug data -V config.cfg


============================ Data file validation ============================
✔ Corpus is loadable
✔ Pipeline can be initialized with data

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
195 training docs
195 evaluation docs
⚠ 195 training examples also in evaluation data
⚠ Low number of examples to train a new pipeline (195)
It's recommended to use at least 2000 examples (minimum 100)

============================== Vocab & Vectors ==============================
ℹ 81046 total word(s) in the data (6841 unique)
10 most common words: ',' (7056), 'and' (3440), '.' (2811), 'in' (1684), '-'
(1655), 'of' (1422), 'to' (1175), 'the' (1105), 'a' (955), 'i' (921)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 0 new label(s), 1 existing label(s)
0 missing value(s) (tokens with '-' label)
Existing: 'ORG'
⚠ 1 entity span(s) with punctuation
✔ Goo

In [179]:
!python -m spacy train config.cfg --output ./output

ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-06-07 16:40:47,736] [INFO] Set up nlp object from config
[2021-06-07 16:40:47,752] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-06-07 16:40:47,757] [INFO] Created vocabulary
[2021-06-07 16:40:47,758] [INFO] Finished initializing nlp object
[2021-06-07 16:40:50,304] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    146.33    0.04    0.02    0.65    0.00
  1     200         34.52   2707.80    1.92   50.00    0.98    0.02
  2     400         66.25    598.75    0.00    0.00    0.00    0.00
  3     600         83.19    547.17   35.29   53.29   26.38    0.35
  4     800 

# TEsting the train model

In [185]:

nlp1 = spacy.load("../training_spacy/output/model-best") #load the best model
doc1 = nlp1("as owner and director of projects at metro infodesign, i proudly promote our unrivaled web design, internet marketing and technical communications services that are ideal for information-intensive businesses that require the fastest, most accurate and scalable means for publishing over the internet and managing large amounts of data, information and content. in addition to serving the small-to-medium size business community, metro infodesign is very supportive to and intimate with web-based cultural and community development projects, in addition to community technology access initiatives. specialties: web design, web development, project management, web portal administration, content development, content management, internet marketing, web promotion, seo, sem, search engine optimization, technical communications, personal branding, clerical methods analysis, business process analysis, meta-data management, joomla, wordpress, drupal, cms, curriculum development, photography, multimedia production., as owner and director of projects at metro infodesign, i proudly promote our unrivaled web development and technical communications services that are ideal for information-intensive businesses that require speed, accuracy, scalability and a means for publishing over the internet and managing large amounts of content. in addition to serving the small-to-medium size business community, metro infodesign is very supportive to and intimate with web-based cultural and community development projects, in addition to community technology access initiatives. specialties: web design, web development, project management, web portal administration, content development, content management, web promotion, technical communications, personal branding, clerical methods analysis, business process analysis, meta-data management, joomla, wordpress, drupal, cms, curriculum development, photography, multimedia production., director of projects and lead photographer at displaymode media | sr. project manager at metro infodesign, director of projects at displaymode media & metro infodesign") # input sample text
for ent in doc1.ents:
    print(ent.label_, ':', ent.text)

ORG : joomla
ORG : wordpress
ORG : drupal
ORG : joomla
ORG : wordpress
ORG : drupal
